# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [1]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)
sum=0
for sqrt_value in generator:
    sum+=sqrt_value

print("What is the sum of the outputs of the generator for limit = 5?",sum)


8.382332347441762


In [3]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1
        if n==14:
          break

# Example usage:
limit = 14
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989


Question 2: What is the 13th number yielded ==> 3.605551275463989

# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [8]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [9]:
import dlt

pipeline = dlt.pipeline(destination='duckdb', dataset_name='load_data')

# we can load any generator to a table at the pipeline destnation as follows:
info = pipeline.run(people_1(),
										table_name="first_table",
										write_disposition="replace")

# the outcome metadata is returned by the load and we can inspect it by printing it.
print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.43 seconds
1 load package(s) were loaded to destination duckdb and into dataset load_data
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1711894083.2667937 is LOADED and contains no failed jobs


In [10]:
import duckdb

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

print("\n sum of peoples age from the first table:")

rides = conn.sql("SELECT SUM(age) FROM first_table").df()
display(rides)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ first_table         │
└─────────────────────┘


 sum of peoples age from the first table:


,sum(age)
0,140.0


In [11]:
info = pipeline.run(people_2(),
										table_name="first_table",
										write_disposition="append")

# the outcome metadata is returned by the load and we can inspect it by printing it.
print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.23 seconds
1 load package(s) were loaded to destination duckdb and into dataset load_data
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1711894153.2101228 is LOADED and contains no failed jobs


In [12]:
print("\n sum of peoples age from all the data:")

age = conn.sql("SELECT SUM(age) FROM first_table").df()
display(age)

print("\n All the data:")

age = conn.sql("SELECT * FROM first_table").df()
display(age)


 sum of peoples age from all the data:


,sum(age)
0,353.0



 All the data:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1711894083.2667937,RfiBQnPB3N7B2A,None
1,2,Person_2,27,City_A,1711894083.2667937,I1bDVXyG+oxJnQ,None
2,3,Person_3,28,City_A,1711894083.2667937,AZwOroOy8eSUng,None
3,4,Person_4,29,City_A,1711894083.2667937,ekU9se5fRPuIdQ,None
4,5,Person_5,30,City_A,1711894083.2667937,XvFy+sxPTEJ/ew,None
5,3,Person_3,33,City_B,1711894153.2101228,GyHen42HElxF2w,Job_3
6,4,Person_4,34,City_B,1711894153.2101228,pf2tWtQMkxVBrw,Job_4
7,5,Person_5,35,City_B,1711894153.2101228,lm77670AYa1g1A,Job_5
8,6,Person_6,36,City_B,1711894153.2101228,Loz8a29S1xzbXQ,Job_6
9,7,Person_7,37,City_B,1711894153.2101228,RqGbgHLcbOsGrQ,Job_7


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


In [13]:
pipeline2 = dlt.pipeline(destination='duckdb', dataset_name='ex3')

# we can load any generator to a table at the pipeline destnation as follows:
info = pipeline2.run(people_1(),
										table_name="first_table",
										write_disposition="replace",
                    primary_key="id")

# the outcome metadata is returned by the load and we can inspect it by printing it.
print(info)

info = pipeline.run(people_2(),
										table_name="first_table",
										write_disposition="merge",
                    primary_key="id")

# the outcome metadata is returned by the load and we can inspect it by printing it.
print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.27 seconds
1 load package(s) were loaded to destination duckdb and into dataset ex3
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1711894551.0428019 is LOADED and contains no failed jobs
Pipeline dlt_colab_kernel_launcher load step completed in 0.49 seconds
1 load package(s) were loaded to destination duckdb and into dataset load_data
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1711894551.8031456 is LOADED and contains no failed jobs


In [14]:
print("\n sum of peoples age from all the data:")

age = conn.sql("SELECT SUM(age) FROM first_table").df()
display(age)

print("\n All the data:")

age = conn.sql("SELECT * FROM first_table").df()
display(age)


 sum of peoples age from all the data:


,sum(age)
0,266.0



 All the data:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1711894083.2667937,RfiBQnPB3N7B2A,None
1,2,Person_2,27,City_A,1711894083.2667937,I1bDVXyG+oxJnQ,None
2,8,Person_8,38,City_B,1711894551.8031456,NCvOGJZ4N0f0IQ,Job_8
3,7,Person_7,37,City_B,1711894551.8031456,NPdupgiH8H78pg,Job_7
4,3,Person_3,33,City_B,1711894551.8031456,CcxwsgrNNA8cYQ,Job_3
5,6,Person_6,36,City_B,1711894551.8031456,BSUCqFJ75Qrg+Q,Job_6
6,5,Person_5,35,City_B,1711894551.8031456,aqJCcycJAJINkg,Job_5
7,4,Person_4,34,City_B,1711894551.8031456,LFGtEBgf3wsNfg,Job_4


# Solution: First make sure that the following modules are installed:

In [4]:
%%capture
!pip install dlt[duckdb]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.8/590.8 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.1 MB/s eta 0:00:00


In [ ]:
# to do: homework :)

Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX